## **Project :-  Amazon Product Analysis**

### Importing Necessary Libraries

In [1]:
# Let's import necessary libraries
import re
import os

import numpy as np
import pandas as pd

# Let's import necessary library for validating the url's
import validators
import time

# SQL server Library
import pyodbc


### Declaring Path & Variables

In [2]:
# Project Folder Path
project_path = r"C:\\Users\\Futurense\\Desktop\\Team Project - Amazon"

# Let's define folder path
folder_path = f"{project_path}\\data"

# Clean File Name
clean_file = "amazon_csv_cleaned.csv"

# driver = "ODBC Driver 18 for SQL Server"
driver = "SQL Server"
server = "DESKTOP-TOEPTEF\SQL_SERVER"
port = 1433

# Database Name
db_name = "AmazonDB"

# table_name
table_name='amazon_product_v1'

### Validate Columns between Files

In [3]:
def get_path_recurs(folder_path: str) :
    """
    Generator to generate the filepath
    
    """
    rows = []
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            p = os.path.join(root, filename)
            yield p

In [4]:
def get_col_names(folder_path: str) -> list:
    """
    Function to get list of column names inside the csv files
    """
    rows = []
    for p in  get_path_recurs(folder_path):
        with open(p, encoding="mbcs") as f:
            row = [p] # Create a list with the file path
            record = f.readlines(1)[0].strip().split(',') # Read the first line, remove leading/trailing whitespaces, and split by commas
            row.append(len(record)) # Append the length of the record list to the row list
            row.extend(record) # Extend the row list with individual values from the record list
            rows.append(row) # Append the row list to the rows list
    return rows

In [5]:
# We are getting column names from all CSV file and storing in DataFrame for comparision

rows = get_col_names(folder_path)
df = pd.DataFrame(rows)

df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11
0,C:\\Users\\Futurense\\Desktop\\Team Project - ...,9,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,None
1,C:\\Users\\Futurense\\Desktop\\Team Project - ...,9,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,None
2,C:\\Users\\Futurense\\Desktop\\Team Project - ...,9,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,None


In [6]:
# We are checking whether columns are same by dropping duplicate columns

df.iloc[:, 2:].drop_duplicates()

,2,3,4,5,6,7,8,9,10,11
0,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,None
10,,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price


In [7]:
df.iloc[10][0]

'C:\\\\Users\\\\Futurense\\\\Desktop\\\\Team Project - Amazon\\data\\Amazon-Products.csv'

**Observations: -**

From above analysis, we could see there is some **discrepancy in col numbers in 10 th row** and all other csv have same columns.

And could see there is empty column in **'Amazon-Product.csv' file**.

On checking we found that It has additional Index col.

On further checking it seems like **'Amazon-Product.csv' already contain complete data**.

But we need to cross check with other csv as well.

#### Check If files were already appended in Amazon-Products

In [8]:
def concat_all_files(file_list: list) -> pd.DataFrame:
    """
    Function to concat all CSV file to DataFrame
    """
    frames = []
    for p in file_list:
        my_df = pd.read_csv(p)
        frames.append(my_df)
    DF = pd.concat(frames)
    return DF

In [9]:
# getting list of files except Amazon-Products

file_list = [p for p in get_path_recurs(folder_path) if "Amazon-Products.csv" not in p]

# Read all csv except "Amazon-Products.csv" and convert to DF
DF = concat_all_files(file_list)

# Reading Amazon-Products
my_df = pd.read_csv(f"{folder_path}\Amazon-Products.csv", index_col=0)

**Note : -** 

**DF** --> Concated CSV files except 'Amazon-Products.csv'

**my_df** --> 'Amazon-Products.csv' to df

In [10]:
# Checking the shape of both DataFrame
print(DF.shape)
print(my_df.shape)

(551585, 9)
(551585, 9)


From above we could see both **DataFrame have same shape**, but we have to verify whether data is all same between them.

In [11]:
def concat_series(obj: pd.Series) -> str:
    """
    Function to concat series values to string
    """
    l = [str(i) for i in obj.values]
    return "".join(l)

**Approach to compare**

To compare data between two DF, we will first concat all rows and create a new column. Then compare concatenated columns between DataFrames, thus if there is any difference in data between them we can findout

In [12]:
 def create_concat_col(df: pd.DataFrame) -> pd.DataFrame:
    """
    Concat all rows to new column and sort them
    """
    df["sorter"] = df.apply(lambda x: concat_series(x), axis=1)
    df = df.sort_values("sorter").reset_index(drop=True)
    return df

In [13]:
my_df = create_concat_col(my_df)
DF = create_concat_col(DF)

In [14]:
# matching only the concated records to find differences
my_df["is_match"] = my_df["sorter"] == DF["sorter"]
DF["is_match"] = my_df["sorter"] == DF["sorter"]

In [15]:
# To Check mismatches
print(f"The No of non-matcing : {len(my_df[~my_df['is_match']])}")

The No of non-matcing : 0


From above analysis, We could see all **data merged except 'Amazon-Products.csv' is same as 'Amazon-Products.csv' file** as per our assumption

So while reading we need to read file only from 'Amazon-Products.csv' file

So from now on we'll consider 'Amazon-Products.csv' as main DataFrame and continue working on it

### Read Data

#### Let's read only Amazon-Products.csv file

In [16]:
df = pd.read_csv(f"{folder_path}\Amazon-Products.csv", index_col=0)

In [17]:
# Let's see top 3 rows of the dataset
df.head(3)

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990"
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990"
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990"


In [18]:
# Let's check the shape of the data

print(f"Shape of the amazon-data.csv file is : - {df.shape}")

Shape of the amazon-data.csv file is : - (551585, 9)


In [19]:
# Let's check about the information of the data

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 551585 entries, 0 to 1103
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            551585 non-null  object
 1   main_category   551585 non-null  object
 2   sub_category    551585 non-null  object
 3   image           551585 non-null  object
 4   link            551585 non-null  object
 5   ratings         375791 non-null  object
 6   no_of_ratings   375791 non-null  object
 7   discount_price  490422 non-null  object
 8   actual_price    533772 non-null  object
dtypes: object(9)
memory usage: 42.1+ MB


**As we can see from above information of the data that : 'ratings', 'no_of_ratings', 'discount_price' & 'actual_price'  columns are having the object datatype which should be integer or float, so we need to check the inconsistency in these columns.**

### Data Consistency

#### Numeric Data Inconsistency

In [20]:
# Let' check the unique values in the ratings ,no_of_ratings, discount_price & actual_price columns


# Let's first work on the ratings column 

df['ratings'].unique()

array(['4.2', '4.0', '4.1', '4.3', '3.9', '3.8', '3.5', nan, '4.6', '3.3',
       '3.4', '3.7', '2.9', '5.0', '4.4', '3.6', '2.7', '4.5', '3.0',
       '3.1', '3.2', '4.8', '4.7', '2.5', '1.0', '2.6', '2.8', '2.3',
       '1.7', 'Get', '1.8', '2.4', '4.9', '2.2', '1.6', '1.9', '2.0',
       '1.4', '2.1', 'FREE', '1.2', '1.3', '1.5', '₹68.99', '₹65', '1.1',
       '₹70', '₹100', '₹99', '₹2.99'], dtype=object)

In [21]:
# Trying to parse rating data

df["corrected_rating"] = pd.to_numeric(df["ratings"], 'coerce')

In [22]:
# To check how many non-NaN rating data converted to NaN

non_corrected = df[df["corrected_rating"].isna() & ~df["ratings"].isna()]

In [23]:
# Let's check the shape of the non_corrected values

non_corrected.shape

(6233, 10)

In [24]:
# Let's see unusual unique value counts, in rating column which must needed to be removed from the column 
# to make that column as a numerical column.

non_corrected["ratings"].unique()

array(['Get', 'FREE', '₹68.99', '₹65', '₹70', '₹100', '₹99', '₹2.99'],
      dtype=object)

In [25]:
# Let's check the value count

non_corrected["ratings"].value_counts()

Get       4852
FREE      1357
₹99         14
₹70          5
₹2.99        2
₹68.99       1
₹65          1
₹100         1
Name: ratings, dtype: int64

**From above data, it is seen than ratings col contains invalid data like rupees, and others
Since those data doesn't make any sense to rating we will be making it null**

In [26]:
def format_numbers(string: str) -> str:
    """
    Convert ₹ and ,   to normal string
    """
    if isinstance(string, float) and np.isnan(string):
        return string
    
    res = string.replace(",", "")
    res = res.replace("₹", "")
    return res

In [27]:
# Trying to parse rating data

df["t_discount_price"] = pd.to_numeric(df["discount_price"].apply(lambda x: format_numbers(x)), 'coerce')
df["t_actual_price"] = pd.to_numeric(df["actual_price"].apply(lambda x: format_numbers(x)), 'coerce')
df["t_no_of_ratings"] = pd.to_numeric(df["no_of_ratings"].apply(lambda x: format_numbers(x)), 'coerce')

In [28]:
# To check how many non-NaN rating data converted to NaN

df[df["t_discount_price"].isna() & ~df["discount_price"].isna()]

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,corrected_rating,t_discount_price,t_actual_price,t_no_of_ratings


In [29]:
# To check how many non-NaN rating data converted to NaN

df[df["t_actual_price"].isna() & ~df["actual_price"].isna()]

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,corrected_rating,t_discount_price,t_actual_price,t_no_of_ratings


In [30]:
# To check how many non-NaN rating data converted to NaN

non_corrected_vals = df[df["t_no_of_ratings"].isna() & ~df["no_of_ratings"].isna()]
non_corrected_vals.head(3)

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,corrected_rating,t_discount_price,t_actual_price,t_no_of_ratings
437,Daikin 1.8 Ton 3 Star Inverter Split Ac (Coppe...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51hn7NHS3T...,https://www.amazon.in/Daikin-Inverter-Copper-F...,Get,Only 2 left in stock.,"₹59,979","₹72,900",NaN,59979.0,72900.0,NaN
473,Master Electronic provides Window AC 3-Star 1....,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41B0LWORAr...,https://www.amazon.in/Master-Electronic-provid...,Get,Only 1 left in stock.,"₹36,999","₹39,999",NaN,36999.0,39999.0,NaN
474,Master Electronic provides 1.5 Ton 3 Star Dust...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/61yVE5P7FU...,https://www.amazon.in/Master-Electronic-provid...,Get,Only 1 left in stock.,"₹36,999","₹39,666",NaN,36999.0,39666.0,NaN


In [31]:
# Let's check the value counts of the column

non_corrected_vals["no_of_ratings"].value_counts()

FREE Delivery by Amazon                           3296
Only 1 left in stock.                             1050
Usually dispatched in 3 to 4 weeks.                847
Only 2 left in stock.                              562
Usually dispatched in 4 to 5 days.                 190
Usually dispatched in 5 to 6 days.                  70
Usually dispatched in 6 to 7 days.                  45
Usually dispatched in 7 to 8 days.                  42
Only 3 left in stock.                               34
Only 5 left in stock.                               29
Only 4 left in stock.                               22
Usually dispatched in 11 to 12 days.                17
Usually dispatched in 2 to 3 weeks.                  8
Usually dispatched in 4 to 5 weeks.                  8
Usually dispatched in 2 to 3 days.                   6
Usually dispatched in 3 to 5 days.                   3
Usually dispatched in 9 to 10 days.                  1
Usually dispatched in 8 to 9 days.                   1
Usually di

**As per above analysis, We could see that there are 6233 records are having invalid no_of_ratings, i.e They contain text data which informs delivery status, So we am considerring them as NULL since they are not realted to rating information.**

In [32]:
def format_numbers(string: str, is_price: bool=False ) -> str:
    '''
    Replacing ',' and '₹' symbols to a proper numberic string.
    Input: string - str - Like discount_price, actual_price, no_of_ratings, ratings.
           is_price - bool - False - Whether a given string is price.?
    Output: It will return a str with numbers.
    '''
    if isinstance(string, float) and np.isnan(string):
        return string
    
    res = string.replace(",", "")
    if is_price:
        res = res.replace("₹", "")
    return res

def covert_numeric_cols(df: pd.DataFrame, cols: list) -> None:
    '''
    It will convert string to number and overwrite the DataFrame.
    Input: df - DataFrame 
           cols - list of columns
    Output: None.
    '''
    for col in cols:
        is_price = False
        if 'price' in col:
            is_price = True
        df[col] = pd.to_numeric(df[col].apply(lambda x: format_numbers(x, is_price)), 'coerce')

In [33]:
cols = ["discount_price",
       "actual_price",
       "no_of_ratings",
       "ratings"]
covert_numeric_cols(df, cols)

#### Categorical Data Inconsistency

**Now lets explore the Categorical columns data to create the function**

In [34]:
# Let's get the number of unique values
print(df["main_category"].nunique())

# Let's get the unique values list
df["main_category"].unique()

20


array(['appliances', 'car & motorbike', 'tv, audio & cameras',
       'sports & fitness', 'grocery & gourmet foods', 'home & kitchen',
       'pet supplies', 'stores', 'toys & baby products', "kids' fashion",
       'bags & luggage', 'accessories', "women's shoes",
       'beauty & health', "men's shoes", "women's clothing",
       'industrial supplies', "men's clothing", 'music',
       'home, kitchen, pets'], dtype=object)

In [35]:
# Let's get the number of unique values
print(df["sub_category"].nunique())

# Let's get the unique values list
df["sub_category"].unique()

112


array(['Air Conditioners', 'All Appliances',
       'All Car & Motorbike Products', 'All Electronics',
       'All Exercise & Fitness', 'All Grocery & Gourmet Foods',
       'All Home & Kitchen', 'All Pet Supplies',
       'All Sports, Fitness & Outdoors', 'Amazon Fashion',
       'Baby Bath, Skin & Grooming', 'Baby Fashion', 'Baby Products',
       'Backpacks', 'Badminton', 'Bags & Luggage', 'Ballerinas',
       'Beauty & Grooming', 'Bedroom Linen', 'Camera Accessories',
       'Cameras', 'Camping & Hiking', 'Car & Bike Care',
       'Car Accessories', 'Car Electronics', 'Car Parts',
       'Cardio Equipment', 'Casual Shoes', 'Clothing',
       'Coffee, Tea & Beverages', 'Cricket', 'Cycling', 'Diapers',
       'Diet & Nutrition', 'Dog supplies', 'Ethnic Wear',
       'Fashion & Silver Jewellery', 'Fashion Sales & Deals',
       'Fashion Sandals', 'Fitness Accessories', 'Football',
       'Formal Shoes', 'Furniture', 'Garden & Outdoors',
       'Gold & Diamond Jewellery', 'Handbags & C

**On checking above string, we could see inconsistency in case and & So we create below function to standardize the text.**

In [36]:
def cat_word_consistency(x:str)->str:
    """
    Text Standardization.
    Input: str
    output: str
    """
    x=x.title().replace("Tv", "TV").replace("&", "And").replace("'S", "'s")
    return x

In [37]:
# Convert Categorical columns to same format

cols = ["main_category",
       "sub_category"]
for col in cols:
    df[col] = df[col].apply(cat_word_consistency)

### Data Validation

**URL Validation**

In [38]:
# Changing URL to remove ref part
df["link"] = df["link"].str.replace('/ref.*', '', regex=True).tolist()

**On checking we could see last part in URL cotains sorting info, which after removal we could find duplicacies. Hence formating the URL to required part
.**

#### Price Dependency Validation

In [39]:
# Checking valid Price data (ActualPrice > 0) and (DiscountPrice >= 0) and (ActualPrice > DiscountPrice)

df["is_price_dependency_valid"] = (df["actual_price"] > 0) \
& (df["discount_price"] >= 0) \
& (df["actual_price"] > df["discount_price"])

In [40]:
df["is_price_dependency_valid"].value_counts()

True     490422
False     61163
Name: is_price_dependency_valid, dtype: int64

**As per above analysis, We could see 61163 have InValid price.**

#### No of Rating Validation

In [41]:
# Checking Valid no_of_rating conditions

df["is_valid_no_of_ratings"] = (df["no_of_ratings"] >= 0)

In [42]:
df["is_valid_no_of_ratings"].value_counts()

True     369558
False    182027
Name: is_valid_no_of_ratings, dtype: int64

**But we could see 182027 rows with Invalid data.**

In [43]:
df[df["is_valid_no_of_ratings"] == False]["no_of_ratings"].unique()


array([nan])

**On checking we could see all the Invalid records are NaN, we will handle NaN in Missing values secitons**

#### Rating Validation

In [44]:
# Checking Rating valid condtion. i.e rating between 1 to 5, 
# also if no-of-rating is zero or Nan then Rating will be 0 which is valid case hence adding that condition as well

df["is_rating_valid"] = (df["ratings"].between(1, 5) | ( (df["ratings"] == 0) ) & ( (df["no_of_ratings"] == 0) ) )

In [45]:
df["is_rating_valid"].value_counts()

True     369558
False    182027
Name: is_rating_valid, dtype: int64

**In above code, we considered if all rating lies between 1 to 5, and if rating is 0 then no_of_ratings also should be zero since no review given.**

In [46]:
df[df["is_rating_valid"] == False]["ratings"].unique()

array([nan])

**On checking same as no_of_rating, we could see all the Invalid records are NaN, we will handle NaN in Missing values secitons.**

**Also we could see the count of NaN is matching between no_of_rating and ratings, which means wherever no_of_rating is missing rating is also missing which is valid scenaario.**

### Duplcates

In [47]:
# Let's check the shape of the data

df.shape

(551585, 16)

In [48]:
# Dropping Duplicates among the same Category
col = ["name", "main_category", "sub_category", "link", "no_of_ratings", "ratings", "actual_price", "discount_price"]
df = df.drop_duplicates(subset=col)

df.shape

(533511, 16)

In [49]:
# Dropping the duplicate data across categories
col = ["name", "link", "no_of_ratings", "ratings", "actual_price", "discount_price"]
df = df.drop_duplicates(subset=col)

df.shape

(473054, 16)

**From above data, I could see the length of data stays same so no duplicate data is present.**

### Missing values

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473054 entries, 0 to 1103
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   name                       473054 non-null  object 
 1   main_category              473054 non-null  object 
 2   sub_category               473054 non-null  object 
 3   image                      473054 non-null  object 
 4   link                       473054 non-null  object 
 5   ratings                    302401 non-null  float64
 6   no_of_ratings              302401 non-null  float64
 7   discount_price             416892 non-null  float64
 8   actual_price               456433 non-null  float64
 9   corrected_rating           302401 non-null  float64
 10  t_discount_price           416892 non-null  float64
 11  t_actual_price             456433 non-null  float64
 12  t_no_of_ratings            302401 non-null  float64
 13  is_price_dependency_valid  4730

**As per above, there is no NaN values in object datatype. But we have made Nan in Numerical columns, so lets check how many zero values are there is each column and check if it is valid or not.**

**Rating Missing Values**

In [51]:
df["is_valid_no_of_ratings"].value_counts()

True     302401
False    170653
Name: is_valid_no_of_ratings, dtype: int64

In [52]:
df["no_of_ratings"].isna().value_counts()

False    302401
True     170653
Name: no_of_ratings, dtype: int64

In [53]:
(df["ratings"].isna()).value_counts()

False    302401
True     170653
Name: ratings, dtype: int64

**On Checking we could see no_of_rating have 182027 Nan, There can be products with No ratings given so we can replace them with zero and change corresponding rating info to zero.**

In [54]:
df[["no_of_ratings", "ratings"]] = df[["no_of_ratings", "ratings"]].fillna(0)

#### Price Missing Values

In [55]:
df["actual_price"].isna().value_counts()

False    456433
True      16621
Name: actual_price, dtype: int64

In [56]:
(df["actual_price"] == 0).value_counts()

False    473052
True          2
Name: actual_price, dtype: int64

In [57]:
df["discount_price"].isna().value_counts()

False    416892
True      56162
Name: discount_price, dtype: int64

In [58]:
df["is_price_dependency_valid"].value_counts()

True     416892
False     56162
Name: is_price_dependency_valid, dtype: int64

In [59]:
# Actual price if it is null/ 0 out of total number of records (%)

17816/551585

0.032299645566866396

**From above analysis, we could see discount have 61163 of Nan values, but it is valid since there are products with No Discounts.**

**But issue is actual_price has 17816 of Nan values, 3 of 0 values, which is not valid, Since they are individual separate products I dont think we can impute data from other products.
Also since 17816 is relatively samller(3%) I am dropping those data.**

In [60]:
# Drop Actual Prive nan/0 actual values

df = df[~df["actual_price"].isna() & (df["actual_price"] > 0)]

In [61]:
# Filling Nan in Discount with zero because discoutns can not be nan, it should be zero

df.loc[:, "discount_price"] = df["discount_price"].fillna(0)

In [63]:
# Removing all Validation columns

cols = ["is_price_dependency_valid",
       "is_rating_valid",
        "is_valid_no_of_ratings",
       "corrected_rating",
       "t_discount_price", 
        "t_actual_price", 
        "t_no_of_ratings"]

df = df.drop(cols, axis=1)

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456431 entries, 0 to 1103
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            456431 non-null  object 
 1   main_category   456431 non-null  object 
 2   sub_category    456431 non-null  object 
 3   image           456431 non-null  object 
 4   link            456431 non-null  object 
 5   ratings         456431 non-null  float64
 6   no_of_ratings   456431 non-null  float64
 7   discount_price  456431 non-null  float64
 8   actual_price    456431 non-null  float64
dtypes: float64(4), object(5)
memory usage: 34.8+ MB


**As we can see from above that all the columns have been transformed, now we are getting a clean data with appropriate datatypes.**

In [68]:
# Let's save the clean data 

df.to_csv(f"{project_path}\\{clean_file}", index=None)

### Let's Load Data to SQL Server

In [69]:
conn_url = f'DRIVER={driver};Server={server};Port={port}'
conn = pyodbc.connect(conn_url)

In [70]:
curs = conn.cursor()

In [71]:
# Refresh DataBase

# switch_db = "use master"
# curs.execute(switch_db)
# curs.commit()

# drop_db = f"Drop database if exists {db_name}"
# curs.execute(drop_db)
# curs.commit()

# create_db = f"create database {db_name}"
# curs.execute(create_db)
# curs.commit()

use_db = f"use {db_name}"
curs.execute(use_db)
curs.commit()

In [68]:
def create_table(df, table_name, curs):
    """
    Function to generate Create Table query
    """

    # DType Mapping For Creating Table
    dtype_map = {
        "object": "varchar(500)",
        "category": "varchar(50)",
        "int64": "int",
        "float64": "decimal(20, 10)",
        "datetime64[ns]": "date",
    }
    
    # Replace / with _ in col names
    pat = "/|-| |\)|\("
    
    col_to_be_rep = [col for col in df.columns if re.search(pat, col)]
    rename_map = {col: re.sub(pat, "_", col) for col in col_to_be_rep}
    df = df.rename(rename_map, axis=1)

    # -------------------------------------Create Table Query Starts------------------------------------------------
    create_table_query = f"""
    create table {table_name}(
    """

    # Define Col Types
    for col in df.columns:
        col_type = f"\t{col} {dtype_map[str(df[col].dtypes)]},\n"
        create_table_query = create_table_query + col_type

    create_table_query = create_table_query + "\n)"
    
    # -------------------------------------Create Table Query Ends------------------------------------------------
    
    # Execute the Create Table command
    curs.execute(create_table_query)
    curs.commit()
    print(f"{table_name} created successfully")
    
    

In [69]:
create_table(df, table_name=table_name, curs=curs)

amazon_product_v1 created successfully


In [74]:
# Inserting the data from CSV to Table using Batch insert
path = f"{project_path}\\{clean_file}"
q = f"""
BULK INSERT dbo.{table_name}
FROM '{path}'
WITH
(
        FORMAT='CSV',
        FIRSTROW=2
);
"""
curs.execute(q)

In [75]:
curs.commit()

In [83]:
# Question 1 : - Which product category has the highest number of items listed?

curs.execute(
"""select
main_category
,count(main_category) as product_cnt
from amazon_product_v1
group by main_category
order by product_cnt desc
""")

query_results = curs.fetchall()

In [91]:
# Let's get the data into a dataframe

df1 = pd.DataFrame.from_records(data = query_results, columns = ['main_category','product_cnt'])

df1

,main_category,product_cnt
0,Accessories,111171
1,Women's Clothing,75812
2,Men's Clothing,73778
3,"TV, Audio And Cameras",66355
4,Men's Shoes,54863
5,Stores,32334
6,Appliances,31326
7,Home And Kitchen,14473
8,Kids' Fashion,13206
9,Sports And Fitness,12253


In [92]:
# Question 2 : - In the "Electronics" category, which sub-category has the highest average rating?

curs.execute("""
select
sub_category
,avg(ratings) as avg_rating
from amazon_product_v1
where main_category = 'appliances'
and ratings != 0 
group by sub_category""")

#ignoring rating with 0 since it means no Rating given, I am calculating avg only for product whose ratings given


query_results2 = curs.fetchall()

In [93]:
# Let's get the data into a dataframe

df2 = pd.DataFrame.from_records(data = query_results2, columns = ['sub_category','avg_rating'])

df2

,sub_category,avg_rating
0,Air Conditioners,3.8256484149
1,Heating And Cooling Appliances,3.6906837374
2,All Appliances,3.9278633623
3,Kitchen And Home Appliances,3.8734088827
4,Refrigerators,3.9256183745
5,Washing Machines,3.9927272727


In [94]:
# Question 3 : -Which product has the highest discount price Among the "Fashion and Silver Jewellery" items?

curs.execute("""
select
name
,discount_price
from amazon_product_v1
where sub_category = 'fashion and silver jewellery'
order by discount_price desc""")

query_results3 = curs.fetchall()

In [95]:
# Let's get the data into a dataframe

df3 = pd.DataFrame.from_records(data = query_results3, columns = ['name','discount_price'])

df3

,name,discount_price
0,Venus Gems Gallery Real Diamond Ring 2 Carat O...,499999.0000000000
1,Krishna Gems Original Diamond VVS1 Clarity 1.8...,204999.0000000000
2,Melorra 22kt 20gm gold coin (916),122254.0000000000
3,P.C. Chandra Jewellers 22k (916) Yellow Gold N...,88171.0000000000
4,MELORRA Women's 22K 22kt Gold Textured Double ...,85623.0000000000
...,...,...
18881,Chandrika Pearls Gems & Jewellers Ganesh Gold ...,0E-10
18882,FOURSEVEN-« Number 7 (Seven) Silver Charm - Fi...,0E-10
18883,Golden Treasure Traditional Gold Plated Long H...,0E-10
18884,Shaya by CaratLane-áHappier than Ever Butterfl...,0E-10


In [96]:
# Question 4 : -In the "Home Improvement" category, which product has the most customer ratings?

curs.execute("""select top 1
name
,no_of_ratings
from amazon_product_v1
where sub_category = 'home improvement'
and ratings != 0 
order by no_of_ratings desc""")


query_results4 = curs.fetchall()

In [97]:
# Let's get the data into a dataframe

df4 = pd.DataFrame.from_records(data = query_results4, columns = ['name','no_of_ratings'])

df4

,name,no_of_ratings
0,Spotzero by Milton Prime Spin Mop with Big Whe...,74689.0000000000


In [98]:
# Question 5 : -What is the average discount percentage across all categories?
curs.execute("""select
main_category
,avg(discount_price/actual_price) * 100.0 as avg_discount_percent
from amazon_product_v1
group by main_category
order by avg_discount_percent desc""")


query_results5 = curs.fetchall()

In [99]:
# Let's get the data into a dataframe

df5 = pd.DataFrame.from_records(data = query_results5, columns = ['main_category','avg_discount_percent'])

df5

,main_category,avg_discount_percent
0,"Home, Kitchen, Pets",76.88399481194023
1,Grocery And Gourmet Foods,60.07094666422307
2,Appliances,57.42057549170733
3,Beauty And Health,57.39536676525559
4,Pet Supplies,55.82575309559997
5,Music,54.66120492304320
6,Industrial Supplies,53.98037476177771
7,Toys And Baby Products,53.23752760023941
8,Kids' Fashion,50.96835209864733
9,"TV, Audio And Cameras",49.31159241668294


In [100]:
# Question 6 : -Which product category has the highest average actual price?
curs.execute("""select
main_category
,avg(actual_price) as avg_actual_price
from amazon_product_v1
group by main_category
order by avg_actual_price desc""")

query_results6 = curs.fetchall()

In [101]:
# Let's get the data into a dataframe

df6 = pd.DataFrame.from_records(data = query_results6, columns = ['main_category','avg_actual_price'])

df6

,main_category,avg_actual_price
0,Home And Kitchen,685834.5640143715
1,Accessories,8136.3061958604
2,Appliances,8132.0863327587
3,Sports And Fitness,8084.1154721292
4,"TV, Audio And Cameras",5725.7190181598
5,Music,5621.8176685934
6,Stores,4959.7710505968
7,"Home, Kitchen, Pets",4468.1176470588
8,Bags And Luggage,4047.9072971348
9,Men's Shoes,3853.4854076517


In [102]:
# Question 7 : among the "Grocery and Gourmet Foods" items, which product has the highest discount percentage?
curs.execute("""select
name
,(discount_price/actual_price) * 100.0 as discount_percent
from amazon_product_v1
where main_category = 'grocery and gourmet foods'
order by discount_percent desc""")

query_results7 = curs.fetchall()

In [103]:
# Let's get the data into a dataframe

df7 = pd.DataFrame.from_records(data = query_results7, columns = ['name','discount_percent'])

df7

,name,discount_percent
0,Hakim Suleman's Olive Vinegar (Zaitoon Sirka) ...,99.93333333333333
1,Jivo Premium Cold Pressed Kachi Ghani Pure Mus...,99.92000000000000
2,"Berries And Nuts Plain Pista Pouch, 500 g",99.90909090909091
3,DXN Product Noni Juice (450ML),99.89473684210526
4,Rungtas Real Gold Special Assam Black Tea - 2 ...,99.88888888888889
...,...,...
3277,Orchard Lane Organic PERI PERI Tomato Ketchup ...,0E-14
3278,"Too Yumm Karare Noodle Masala, 75g",0E-14
3279,"Amazon Brand - Vedaka Fennel Seeds (Saunf), 100 g",0E-14
3280,"Sumeru Chicken Malai Seekh Kebab, 400 g",0E-14


In [104]:
# Question 8 :In the "Sports Fitness and Outdoors" category, which product has the highest number of customer ratings?

curs.execute("""select
name
,no_of_ratings
from amazon_product_v1
where main_category = 'sports and fitness'
order by no_of_ratings desc""")

query_results8 = curs.fetchall()

In [106]:
# Let's get the data into a dataframe

df8 = pd.DataFrame.from_records(data = query_results8, columns = ['name','no_of_ratings'])

df8

,name,no_of_ratings
0,AmazonBasics 13mm Extra Thick Yoga and Exercis...,74871.0000000000
1,"AmazonBasics Neoprene Dumbbells, Set of 2",44728.0000000000
2,Kore PVC 10-40 Kg Home Gym Set with One 3 Ft C...,33078.0000000000
3,Campus Men's OXYFIT (N) Walking Shoe,30227.0000000000
4,Fitness Mantra-« EVA Yoga Mat for Gym Workout ...,21710.0000000000
...,...,...
12248,Jimmy Sports Men's Regular Fit Trackpants|Black,0E-10
12249,Fashiol Women Seamless Bra Full Coverage Breat...,0E-10
12250,PALAY-« Bucket Hat for Women Men Reversible Fa...,0E-10
12251,Skechers Arch FIT - Big Appeal Women's Running...,0E-10


In [107]:
# Question 9 :.Among the "Health and Personal Care" items, which product has the highest average rating?

curs.execute("""select
name
,avg(ratings) as avg_rating
from amazon_product_v1
where sub_category = 'Health and Personal Care'
group by name
order by avg_rating desc""")

query_results9 = curs.fetchall()

In [108]:
# Let's get the data into a dataframe

df9 = pd.DataFrame.from_records(data = query_results9, columns = ['name','avg_rating'])

df9

,name,avg_rating
0,JIPL Neck & Shoulder Relaxer for TMJ Pain | Ne...,5.0000000000
1,"Bosom Breast Ayurvedic Massage Oil For Women,A...",5.0000000000
2,SAJANI Washing Machine Cleaner Descaler 12 Pac...,5.0000000000
3,"PGR Surface & Floor Cleaner Liquid, Floral Fra...",5.0000000000
4,RAWLS Hydrating Under Eye Cream Gel/Enriched w...,5.0000000000
...,...,...
1076,RJGUDDU 20g Warts Remover Cream Extract Skin F...,0E-10
1077,"Fresh Aloe Vera, 1 Pc",0E-10
1078,Accu-Chek Active 50 Test Strips + 2 packs of l...,0E-10
1079,Neurobion Forte - Strip of 30 Tablets,0E-10


In [109]:
# Question 10 :In the "Kitchen and Dining" category, which sub-category has the highest number of items listed?

curs.execute("""select
sub_category
,count(sub_category) as prod_cnt
from amazon_product_v1
where main_category = 'home and kitchen'
group by sub_category
order by prod_cnt desc;""")


query_results10 = curs.fetchall()


In [110]:
# Let's get the data into a dataframe

df10 = pd.DataFrame.from_records(data = query_results10, columns = ['sub_category','prod_cnt'])

df10

,sub_category,prod_cnt
0,Home Furnishing,1411
1,Furniture,1301
2,Home D+¬cor,1268
3,Sewing And Craft Supplies,1259
4,Indoor Lighting,1235
5,All Home And Kitchen,1220
6,Home Storage,1220
7,Bedroom Linen,1214
8,Kitchen And Dining,1197
9,Garden And Outdoors,1098
